In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
from pyFTS.benchmarks import Measures

## Auxiliary Functions

In [2]:
def normalize(df):
    mindf = df.min()
    maxdf = df.max()
    return (df-mindf)/(maxdf-mindf)

In [3]:
def denormalize(norm, _min, _max):
    return [(n * (_max-_min)) + _min for n in norm]

In [4]:
def save_obj(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Load Dataset

In [5]:
#Set target and input variables 
target_station = 'DHHL_3'

#All neighbor stations with residual correlation greater than .90
neighbor_stations_90 = ['DHHL_3',  'DHHL_4','DHHL_5','DHHL_10','DHHL_11','DHHL_9','DHHL_2', 'DHHL_6','DHHL_7','DHHL_8']

In [6]:
df = pd.read_pickle("df_oahu.pkl")

In [7]:
## Remove columns with many corrupted or missing values
df.drop(columns=['AP_1', 'AP_7'], inplace=True)

In [8]:
# Get data form the interval of interest
interval = ((df.index >= '2010-06') & (df.index < '2010-08'))
df = df.loc[interval]

In [9]:
#Normalize Data

# Save Min-Max for Denorm
min_raw = df[target_station].min()

max_raw = df[target_station].max()

# Perform Normalization
norm_df = normalize(df)

In [21]:
def get_final_forecast(forecast_list):
    forecast_final = []
    
    for fcst in forecast_list:
        forecast_final.append(denormalize(fcst, min_raw, max_raw))
        
    return forecast_final

## Create Rolling window indexes

- Training: 4 weeks
- Validation: 1 week
- Test: 1 week

Roliing daily

In [10]:
import datetime

In [11]:
def getRollingWindow(index):
    pivot = index
    train_start = pivot.strftime('%Y-%m-%d')
    pivot = pivot + datetime.timedelta(days=27)
    train_end = pivot.strftime('%Y-%m-%d')

    pivot = pivot + datetime.timedelta(days=1)
    validation_start = pivot.strftime('%Y-%m-%d')
    pivot = pivot + datetime.timedelta(days=6)
    validation_end = pivot.strftime('%Y-%m-%d')

    pivot = pivot + datetime.timedelta(days=1)
    test_start = pivot.strftime('%Y-%m-%d')
    pivot = pivot + datetime.timedelta(days=6)
    test_end = pivot.strftime('%Y-%m-%d')
    
    return train_start, train_end, validation_start, validation_end, test_start, test_end

In [12]:
def calculate_rolling_error(cv_name, df, forecasts, order_list):
    cv_results = pd.DataFrame(columns=['Split', 'RMSE', 'SMAPE', 'U'])

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    for i in np.arange(len(forecasts)):

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        test = df[test_start : test_end]
    
        yhat = forecasts[i]
        order = order_list[i]
        
        rmse = Measures.rmse(test[target_station].iloc[order:], yhat[:-1])
        
        smape = Measures.smape(test[target_station].iloc[order:], yhat[:-1])
        
        u = Measures.UStatistic(test[target_station].iloc[order:], yhat[:-1])
       
        res = {'Split' : index.strftime('%Y-%m-%d') ,'RMSE' : rmse, 'SMAPE' : smape, 'U' : u}
        cv_results = cv_results.append(res, ignore_index=True)
        cv_results.to_csv(cv_name+".csv")        

        index = index + datetime.timedelta(days=7)

## Persistence

In [13]:
def persistence_forecast(train, test, step):
    predictions = []
    
    for t in np.arange(0,len(test), step):
        yhat = [test.iloc[t]]  * step
        predictions.extend(yhat)
        
    return predictions

In [14]:
def rolling_cv_persistence(df, step):

    forecasts = []
    lags_list = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]
    
        # Concat train & validation for test
        train = train.append(validation)
        yhat = persistence_forecast(train[target_station], test[target_station], step)        
        
        lags_list.append(1)
        forecasts.append(yhat)

    return forecasts, lags_list

In [22]:
forecasts, order_list = rolling_cv_persistence(norm_df, 1)
forecasts_final = get_final_forecast(forecasts)

Index:  2010-06-01
Index:  2010-06-08
Index:  2010-06-15
Index:  2010-06-22


In [20]:
calculate_rolling_error("rolling_cv_oahu_raw_persistence", df, forecasts_final, order_list)

## SARIMA

In [15]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
import sys

/Users/cseveriano/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
def evaluate_SARIMA_models(test_name, train, validation, parameters_list, period_length):

    sarima_results = pd.DataFrame(columns=['Order','RMSE'])
    best_score, best_cfg = float("inf"), None

    for param in parameters_list:
        arima_order = (param[0],param[1],param[2])
        sarima_order = (param[3],param[4],param[5],period_length)
        print('Testing SARIMA%s %s ' % (str(arima_order),str(sarima_order)))
        try:
            fcst = sarima_forecast(train, validation, arima_order, sarima_order)
            rmse = Measures.rmse(validation.values, fcst)
            
            if rmse < best_score:
                best_score, best_cfg = rmse, (arima_order, sarima_order)

            res = {'Parameters' : str(param) ,'RMSE' : rmse}
            print('SARIMA%s %s RMSE=%.3f' % (str(arima_order),str(sarima_order),rmse))
            sarima_results = sarima_results.append(res, ignore_index=True)
            sarima_results.to_csv(test_name+".csv")
        except:
            print(sys.exc_info())
            print('Invalid model%s %s ' % (str(arima_order),str(sarima_order)))
            continue
    
    print('Best SARIMA(%s) RMSE=%.3f' % (best_cfg, best_score))
    return best_cfg

In [28]:
def sarima_forecast(train, test, arima_order, sarima_order):

    predictions = []
    window_size = sarima_order[3] * 5
    
    history = list(train.iloc[-window_size:])

    model = SARIMAX(history, order=arima_order, seasonal_order=sarima_order,enforce_invertibility=False,enforce_stationarity=False)
    model_fit = model.fit(disp=True,enforce_invertibility=False, maxiter=10)

    
    #save the state parameter
    est_params = model_fit.params
    est_state = model_fit.predicted_state[:, -1]
    est_state_cov = model_fit.predicted_state_cov[:, :, -1]

    st = 0
        
    for t in np.arange(1,len(test)+1,1):
        obs = test.iloc[st:t].values
        history.extend(obs)
        history = history[-window_size:]
        
        mod_updated = SARIMAX(history, order=arima_order, seasonal_order=sarima_order,enforce_invertibility=False,enforce_stationarity=False)
        mod_updated.initialize_known(est_state, est_state_cov)
        mod_frcst = mod_updated.smooth(est_params)
        
        yhat = mod_frcst.forecast(1)   
        predictions.extend(yhat)
            
        est_params = mod_frcst.params
        est_state = mod_frcst.predicted_state[:, -1]
        est_state_cov = mod_frcst.predicted_state_cov[:, :, -1]
            
        st = t
                
    return predictions

In [59]:
def rolling_cv_SARIMA(df, step):

#    p_values = [0,1,2]
#    d_values = [0,1]
#    q_values = [0,1,2]
#    P_values = [0,1]
#    D_Values = [0,1]
#    Q_Values = [0,1]

    p_values = [2]
    d_values = [1]
    q_values = [2]
    P_values = [1]
    D_Values = [1]
    Q_Values = [1]

    parameters = product(p_values, d_values, q_values, P_values, D_Values, Q_Values)
    parameters_list = list(parameters)
    period_length = 61 #de 5:00 as 20:00

    
    forecasts = []
    lags_list = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]
    
        # Perform grid search
        #(arima_params, sarima_params) = evaluate_SARIMA_models("nested_test_sarima_oahu", train[target_station], validation[target_station], parameters_list, period_length)
        arima_params = (2, 1, 2)
        sarima_params = (1, 1, 1, 61)
        
        # Concat train & validation for test
        train = train.append(validation)
        yhat = sarima_forecast(train[target_station], test[target_station], arima_params, sarima_params)        
        
        lags_list.append(1)
        forecasts.append(yhat)

    return forecasts, lags_list

In [30]:
forecasts, order_list = rolling_cv_SARIMA(norm_df, 1)

Index:  2010-06-01
Index:  2010-06-08


/Users/cseveriano/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


KeyboardInterrupt: 

In [ ]:
forecasts_final, order_list = get_final_forecast(forecasts)

In [ ]:
calculate_rolling_error("rolling_cv_oahu_raw_sarima", df, forecasts_final, order_list)

## Vector Autoregressive

In [23]:
from statsmodels.tsa.api import VAR, DynamicVAR

/Users/cseveriano/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [24]:
def evaluate_VAR_models(test_name, train, validation,target, maxlags_list):
    var_results = pd.DataFrame(columns=['Order','RMSE'])
    best_score, best_cfg, best_model = float("inf"), None, None
    
    for lgs in maxlags_list:
        model = VAR(train)
        results = model.fit(maxlags=lgs, ic='aic')
        
        order = results.k_ar
        forecast = []

        for i in range(len(validation)-order) :
            forecast.extend(results.forecast(validation.values[i:i+order],1))

        forecast_df = pd.DataFrame(columns=validation.columns, data=forecast)
        rmse = Measures.rmse(validation[target].iloc[order:], forecast_df[target].values)

        if rmse < best_score:
            best_score, best_cfg, best_model = rmse, order, results

        res = {'Order' : str(order) ,'RMSE' : rmse}
        print('VAR (%s)  RMSE=%.3f' % (str(order),rmse))
        var_results = var_results.append(res, ignore_index=True)
        var_results.to_csv(test_name+".csv")
        
    print('Best VAR(%s) RMSE=%.3f' % (best_cfg, best_score))
    return best_model

In [25]:
def var_forecast(train, test, target, order, step):
    model = VAR(train.values)
    results = model.fit(maxlags=order)
    lag_order = results.k_ar
    print("Lag order:" + str(lag_order))
    forecast = []

    for i in np.arange(0,len(test)-lag_order+1,step) :
        forecast.extend(results.forecast(test.values[i:i+lag_order],step))

    forecast_df = pd.DataFrame(columns=test.columns, data=forecast)
    return forecast_df[target].values

In [26]:
def nested_cv_var(df, step):
    maxlags_list = [1,2,4,6,8,10,20,40]
    forecasts = []
    order_list = []
    
    for i in np.arange(n_folds):
        train = getNestedData(df, train_inds[i])
        validation = getNestedData(df, val_inds[i])
        test = getNestedData(df, test_inds[i])
    
        # Perform grid search
        best_model = evaluate_VAR_models("nested_test_var_oahu", train[neighbor_stations_90], validation[neighbor_stations_90],target_station, maxlags_list)

        # Concat train & validation for test
        train = train.append(validation)
        order = best_model.k_ar
        yhat = var_forecast(train[neighbor_stations_90], test[neighbor_stations_90], target_station, order, step)
        
        order_list.append(order)
        forecasts.append(yhat)
    
    return forecasts, order_list

In [27]:
def rolling_cv_var(df, step):
    maxlags_list = [1,2,4,6,8,10,20,40]
    forecasts = []
    order_list = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]
    
        # Perform grid search
        best_model = evaluate_VAR_models("nested_test_var_oahu", train[neighbor_stations_90], validation[neighbor_stations_90],target_station, maxlags_list)

        # Concat train & validation for test
        train = train.append(validation)
        order = best_model.k_ar
        yhat = var_forecast(train[neighbor_stations_90], test[neighbor_stations_90], target_station, order, step)
        
        order_list.append(order)
        forecasts.append(yhat)

    return forecasts, order_list

In [28]:
forecasts, order_list = rolling_cv_var(norm_df, 1)

Index:  2010-06-01
VAR (1)  RMSE=0.093
VAR (2)  RMSE=0.092
VAR (4)  RMSE=0.094
VAR (6)  RMSE=0.095
VAR (8)  RMSE=0.097
VAR (10)  RMSE=0.097
VAR (10)  RMSE=0.097
VAR (10)  RMSE=0.097
Best VAR(2) RMSE=0.092
Lag order:2
Index:  2010-06-08
VAR (1)  RMSE=0.120
VAR (2)  RMSE=0.118
VAR (4)  RMSE=0.118
VAR (6)  RMSE=0.119
VAR (8)  RMSE=0.118
VAR (8)  RMSE=0.118
VAR (8)  RMSE=0.118
VAR (8)  RMSE=0.118
Best VAR(8) RMSE=0.118
Lag order:8
Index:  2010-06-15
VAR (1)  RMSE=0.121
VAR (2)  RMSE=0.123
VAR (4)  RMSE=0.123
VAR (6)  RMSE=0.123
VAR (8)  RMSE=0.125
VAR (8)  RMSE=0.125
VAR (8)  RMSE=0.125
VAR (8)  RMSE=0.125
Best VAR(1) RMSE=0.121
Lag order:1
Index:  2010-06-22
VAR (1)  RMSE=0.104
VAR (2)  RMSE=0.103
VAR (4)  RMSE=0.105
VAR (6)  RMSE=0.106
VAR (8)  RMSE=0.107
VAR (8)  RMSE=0.107
VAR (8)  RMSE=0.107
VAR (8)  RMSE=0.107
Best VAR(2) RMSE=0.103
Lag order:2


In [29]:
forecasts_final = get_final_forecast(forecasts)

In [30]:
calculate_rolling_error("rolling_cv_oahu_raw_var", df, forecasts_final, order_list)

## High Order FTS

In [31]:
from pyFTS.partitioners import Grid, Entropy, Util as pUtil
from pyFTS.models import hofts

In [32]:
def evaluate_hofts_models(test_name, train, validation, partitioners_list, order_list, partitions_list):
    
    hofts_results = pd.DataFrame(columns=['Partitioner','Partitions','Order','RMSE'])
    best_score, best_cfg, best_model = float("inf"), None, None

    for _partitioner in partitioners_list:
        for _order in order_list:
            for npartitions in partitions_list:
                fuzzy_sets = _partitioner(data=train.values, npart=npartitions)
                model_simple_hofts = hofts.HighOrderFTS(order=_order)

                model_simple_hofts.fit(train.values, order=_order, partitioner=fuzzy_sets)
                
                forecast = model_simple_hofts.predict(validation.values)
                rmse = Measures.rmse(validation.iloc[_order:], forecast[:-1])

                if rmse < best_score:
                    best_score, best_cfg = rmse, (_order,npartitions,_partitioner)
                    best_model = model_simple_hofts

                res = {'Partitioner':str(_partitioner), 'Partitions':npartitions, 'Order' : str(_order) ,'RMSE' : rmse}
                print('HOFTS %s - %s - %s  RMSE=%.3f' % (str(_partitioner), npartitions, str(_order),rmse))
                hofts_results = hofts_results.append(res, ignore_index=True)
                hofts_results.to_csv(test_name+".csv")

    print('Best HOFTS(%s) RMSE=%.3f' % (best_cfg, best_score))
    
    return best_cfg

In [33]:
def hofts_forecast(train_df, test_df, _order, _partitioner, _npartitions):
    
    fuzzy_sets = _partitioner(data=train_df.values, npart=_npartitions)
    model_simple_hofts = hofts.HighOrderFTS()
    

    model_simple_hofts.fit(train_df.values, order=_order, partitioner=fuzzy_sets)

    
    forecast = model_simple_hofts.predict(test_df.values)

    return forecast

In [36]:
def rolling_cv_hofts(df, step):
    
    partitioners_list = [Grid.GridPartitioner, Entropy.EntropyPartitioner]
    eval_order_list = np.arange(1,3)
    partitions_list = np.arange(80,100,10)
    
    forecasts = []
    order_list = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=1)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]

        # Perform grid search
        (order,nparts,partitioner) = evaluate_hofts_models("nested_eval_hofts_oahu", train[target_station], validation[target_station], partitioners_list, eval_order_list, partitions_list)

        # Concat train & validation for test
        train = train.append(validation)

        # Perform forecast
        yhat = hofts_forecast(train[target_station], test[target_station], order, partitioner, nparts)
        
        order_list.append(order)
        forecasts.append(yhat)

    return forecasts, order_list

In [37]:
forecasts, order_list = rolling_cv_hofts(norm_df, 1)

Index:  2010-06-01
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 80 - 1  RMSE=0.099
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 90 - 1  RMSE=0.097
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 80 - 2  RMSE=0.105
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 90 - 2  RMSE=0.102
HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 80 - 1  RMSE=0.105
HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 90 - 1  RMSE=0.105


/Users/cseveriano/anaconda3/lib/python3.6/site-packages/pyFTS/common/flrg.py:52: RuntimeWarning: Mean of empty slice
  self.midpoint = np.nanmean(self.get_midpoints(sets))
/Users/cseveriano/anaconda3/lib/python3.6/site-packages/pyFTS/models/hofts.py:136: RuntimeWarning: Mean of empty slice
  ret.append(np.nanmean(tmp))


HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 80 - 2  RMSE=0.102
HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 90 - 2  RMSE=0.102
Best HOFTS((1, 90, <class 'pyFTS.partitioners.Grid.GridPartitioner'>)) RMSE=0.097
Index:  2010-06-02
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 80 - 1  RMSE=0.100
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 90 - 1  RMSE=0.099
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 80 - 2  RMSE=0.107
HOFTS <class 'pyFTS.partitioners.Grid.GridPartitioner'> - 90 - 2  RMSE=0.102
HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 80 - 1  RMSE=0.105
HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 90 - 1  RMSE=0.105
HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 80 - 2  RMSE=0.104
HOFTS <class 'pyFTS.partitioners.Entropy.EntropyPartitioner'> - 90 - 2  RMSE=0.104
Best HOFTS((1, 90, <class 'pyFTS.partitioners.Grid.GridPartitioner'>)) RMSE=0.099
Index:  201

TypeError: 'NoneType' object is not iterable

In [ ]:
forecasts_final = get_final_forecast(forecasts)

In [ ]:
calculate_rolling_error("rolling_cv_oahu_raw_hofts", df, forecasts_final, order_list)

## Conditional Variance FTS

In [55]:
from pyFTS.models.nonstationary import cvfts
from pyFTS.models.nonstationary import partitioners as nspartitioners

In [65]:
def evaluate_cvfts_models(test_name, train, validation, partitions_list):
    
    cvfts_results = pd.DataFrame(columns=['Partitions','RMSE'])
    best_score, best_cfg, best_model = float("inf"), None, None

    for npartitions in partitions_list:
                
        fuzzy_sets =  nspartitioners.PolynomialNonStationaryPartitioner(data=train.values, part=Grid.GridPartitioner(data=train.values, npart=npartitions), degree=2)
                
        model_cvfts = cvfts.ConditionalVarianceFTS()
        model_cvfts.fit(train.values, parameters=1, partitioner=fuzzy_sets, num_batches=1000)
                                
        forecast = model_cvfts.predict(validation.values)
        rmse = Measures.rmse(validation.iloc[1:], forecast[:-1])

        if rmse < best_score:
            best_score, best_cfg = rmse, npartitions
            best_model = model_cvfts

        res = {'Partitions':npartitions, 'RMSE' : rmse}
        print('CVFTS %s -  RMSE=%.3f' % (npartitions, rmse))
        cvfts_results = cvfts_results.append(res, ignore_index=True)
        cvfts_results.to_csv(test_name+".csv")

    print('Best CVFTS(%s) RMSE=%.3f' % (best_cfg, best_score))
    
    return best_cfg

In [66]:
def cvfts_forecast(train, test, _partitions):
    
    fuzzy_sets =  nspartitioners.PolynomialNonStationaryPartitioner(data=train.values, part=Grid.GridPartitioner(data=train.values, npart=_partitions), degree=2)
                    
    model_cvfts = cvfts.ConditionalVarianceFTS()
    model_cvfts.fit(train.values, parameters=1, partitioner=fuzzy_sets, num_batches=1000)

    forecast = model_cvfts.predict(test.values)

    return forecast

In [24]:
def rolling_cv_cvfts(df, step):
    
    partitions_list = np.arange(80,100,10)
    
    forecasts = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]

        # Perform grid search
        nparts = evaluate_cvfts_models("nested_eval_cvfts_oahu", train[target_station], validation[target_station], partitions_list)

        # Concat train & validation for test
        train = train.append(validation)

        # Perform forecast
        yhat = cvfts_forecast(train[target_station], test[target_station],nparts)
        
        order_list.append(1)
        forecasts.append(yhat)

    return forecasts, order_list

In [67]:
forecasts, order_list = rolling_cv_cvfts(norm_df, 1)

Index:  2010-06-01
CVFTS 10 -  RMSE=0.049
CVFTS 20 -  RMSE=0.037
CVFTS 30 -  RMSE=0.032
CVFTS 40 -  RMSE=0.032
CVFTS 50 -  RMSE=0.032
CVFTS 60 -  RMSE=0.031
CVFTS 70 -  RMSE=0.031
CVFTS 80 -  RMSE=0.031
CVFTS 90 -  RMSE=0.031
Best CVFTS(90) RMSE=0.031
Index:  2010-06-02


/Users/cseveriano/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: ComplexWarning: Casting complex values to real discards the imaginary part


CVFTS 10 -  RMSE=0.052
CVFTS 20 -  RMSE=0.036
CVFTS 30 -  RMSE=0.032
CVFTS 40 -  RMSE=0.032
CVFTS 50 -  RMSE=0.032
CVFTS 60 -  RMSE=0.031
CVFTS 70 -  RMSE=0.031
CVFTS 80 -  RMSE=0.031
CVFTS 90 -  RMSE=0.031
Best CVFTS(90) RMSE=0.031


/Users/cseveriano/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: ComplexWarning: Casting complex values to real discards the imaginary part


Index:  2010-06-03
CVFTS 10 -  RMSE=0.051
CVFTS 20 -  RMSE=0.037
CVFTS 30 -  RMSE=0.032
CVFTS 40 -  RMSE=0.032
CVFTS 50 -  RMSE=0.031
CVFTS 60 -  RMSE=0.031
CVFTS 70 -  RMSE=0.031
CVFTS 80 -  RMSE=0.031
CVFTS 90 -  RMSE=0.031
Best CVFTS(70) RMSE=0.031
Index:  2010-06-04
CVFTS 10 -  RMSE=0.049
CVFTS 20 -  RMSE=0.035
CVFTS 30 -  RMSE=0.032
CVFTS 40 -  RMSE=0.032
CVFTS 50 -  RMSE=0.031
CVFTS 60 -  RMSE=0.031
CVFTS 70 -  RMSE=0.031
CVFTS 80 -  RMSE=0.031
CVFTS 90 -  RMSE=0.031
Best CVFTS(90) RMSE=0.031
Index:  2010-06-05
CVFTS 10 -  RMSE=0.050


KeyboardInterrupt: 

In [ ]:
forecasts_final = get_final_forecast(forecasts)

In [ ]:
calculate_rolling_error("rolling_cv_oahu_raw_cvfts", df, forecasts_final, order_list)

## Clustered Multivariate FTS

In [32]:
!pip install -U git+https://github.com/cseveriano/spatio-temporal-forecasting

  Cloning https://github.com/cseveriano/spatio-temporal-forecasting to /private/var/folders/13/t7d8w0nd0hv6w9_p2rntvym00000gr/T/pip-req-build-dtzphc34
  Running setup.py bdist_wheel for spatio-temporal-forecasting ... done
  Stored in directory: /private/var/folders/13/t7d8w0nd0hv6w9_p2rntvym00000gr/T/pip-ephem-wheel-cache-i5nysv4z/wheels/d2/1f/6f/439795864246039ef36c6a3c88edf7935c803c2cf97133066a
Successfully built spatio-temporal-forecasting
  Found existing installation: spatio-temporal-forecasting 0.1
    Uninstalling spatio-temporal-forecasting-0.1:
      Successfully uninstalled spatio-temporal-forecasting-0.1
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [33]:
from models import KMeansPartitioner
from models import sthofts

In [53]:
def cmvfts_forecast(train_df, test_df, target, _order, npartitions):
    
    _partitioner = KMeansPartitioner.KMeansPartitioner(data=train_df.values, npart=npartitions, batch_size=1000, init_size=npartitions*3)
    model_sthofts = sthofts.SpatioTemporalHighOrderFTS(membership_threshold=0.6)
    
    model_sthofts.fit(train_df.values, num_batches=100, order=_order, partitioner=_partitioner)
    forecast = model_sthofts.predict(test_df.values)
    forecast_df = pd.DataFrame(data=forecast, columns=test_df.columns)
    return forecast_df[target].values

In [46]:
def evaluate_cmvfts_models(test_name, train, validation, order_list, partitions_list):
    
    cmvfts_results = pd.DataFrame(columns=['Partitions','Order','RMSE'])
    best_score, best_cfg = float("inf"), None

    for _order in order_list:
        for npartitions in partitions_list:
            
            forecast = cmvfts_forecast(train, validation, target_station, _order, npartitions)
            rmse = Measures.rmse(validation[target_station].iloc[_order:], forecast[:-1])

            if rmse < best_score:
                best_score, best_cfg = rmse, (_order,npartitions)

            res = {'Partitions':npartitions, 'Order' : str(_order) ,'RMSE' : rmse}
            print('CMVFTS %s - %s  RMSE=%.3f' % (npartitions, str(_order),rmse))
            cmvfts_results = cmvfts_results.append(res, ignore_index=True)
            cmvfts_results.to_csv(test_name+".csv")

    print('Best CMVFTS(%s) RMSE=%.3f' % (best_cfg, best_score))
    
    return best_cfg

In [47]:
def rolling_cv_cmvfts(df, step):
    
    eval_order_list = np.arange(1,3)
    partitions_list = np.arange(10,100,10)
    
    forecasts = []
    order_list = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]

        # Perform grid search
        (order,nparts) = evaluate_cmvfts_models("nested_eval_cmvfts_oahu", train[neighbor_stations_90], validation[neighbor_stations_90], eval_order_list, partitions_list)

        # Concat train & validation for test
        train = train.append(validation)

        # Perform forecast
        yhat = cmvfts_forecast(train[neighbor_stations_90], test[neighbor_stations_90], order, nparts)
        
        order_list.append(order)
        forecasts.append(yhat)

    return forecasts, order_list

In [54]:
forecasts, order_list = rolling_cv_cmvfts(norm_df, 1)

Index:  2010-06-01
CMVFTS 10 - 1  RMSE=0.046
CMVFTS 20 - 1  RMSE=0.036
CMVFTS 30 - 1  RMSE=0.034
CMVFTS 40 - 1  RMSE=0.034
CMVFTS 50 - 1  RMSE=0.034


KeyboardInterrupt: 

In [ ]:
forecasts_final = get_final_forecast(forecasts)
calculate_rolling_error("rolling_cv_oahu_raw_cmvfts", df, forecasts_final, order_list)

## LSTM - Multivariate

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.constraints import maxnorm

Using TensorFlow backend.


In [17]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (  t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [44]:
def lstm_multi_forecast(train_df, test_df, _order, _steps, _neurons, _epochs):

    
    nfeat = len(train_df.columns)
    nlags = _order
    nsteps = _steps
    nobs = nlags * nfeat
    
    train_reshaped_df = series_to_supervised(train_df, n_in=nlags, n_out=nsteps)
    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
    train_X = train_X.reshape((train_X.shape[0], nlags, nfeat))

    test_reshaped_df = series_to_supervised(test_df, n_in=nlags, n_out=nsteps)
    test_X, test_Y = test_reshaped_df.iloc[:,:nobs].values, test_reshaped_df.iloc[:,-nfeat].values
    test_X = test_X.reshape((test_X.shape[0], nlags, nfeat))
    
    # design network
    model = Sequential()
    model.add(LSTM(_neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')

    # fit network
    model.fit(train_X, train_Y, epochs=_epochs, batch_size=1000, verbose=False, shuffle=False)
    
    forecast = model.predict(test_X)
    
    fcst = [f[0] for f in forecast]

    return fcst

In [52]:
def evaluate_multivariate_lstm_models(test_name, train_df, validation_df, neurons_list, order_list, epochs_list):
    
    lstm_results = pd.DataFrame(columns=['Neurons','Order','Epochs','RMSE'])
    best_score, best_cfg = float("inf"), None
    
    nfeat = len(train_df.columns)
    nsteps = 1
    
    for _neurons in neurons_list:
        for _order in order_list:
            for epochs in epochs_list:
                    
                    nobs = nfeat * _order
                    
                    train_reshaped_df = series_to_supervised(train_df, n_in=_order, n_out=nsteps)
                    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
                    train_X = train_X.reshape((train_X.shape[0], _order, nfeat))                    
                    
                    val_reshaped_df = series_to_supervised(validation_df, n_in=_order, n_out=nsteps)
                    validation_X, validation_Y = val_reshaped_df.iloc[:,:nobs].values, val_reshaped_df.iloc[:,-nfeat].values
                    validation_X = validation_X.reshape((validation_X.shape[0], _order, nfeat))
                    
                    # design network
                    model = Sequential()
                    model.add(LSTM(_neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
                    model.add(Dense(1))
                    model.compile(loss='mae', optimizer='adam')

                    # fit network
                    history = model.fit(train_X, train_Y, epochs=epochs, batch_size=1000, verbose=False, shuffle=False)
                    forecast = model.predict(validation_X)
                    fcst = [f[0] for f in forecast]
                    
                    
                    rmse = Measures.rmse(validation_Y, fcst)
                    #rmse = math.sqrt(mean_squared_error(validation_Y, forecast))
                    
                    params = (_neurons, _order,epochs)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, params

                    res = {'Neurons':_neurons, 'Order':_order, 'Epochs' : epochs ,'RMSE' : rmse}
                    print('LSTM %s  RMSE=%.3f' % (params,rmse))
                    lstm_results = lstm_results.append(res, ignore_index=True)
                    lstm_results.to_csv(test_name+".csv")

    print('Best LSTM(%s) RMSE=%.3f' % (best_cfg, best_score))
    return best_cfg

In [53]:
def rolling_cv_lstm_multi(df, step):
    
    neurons_list = np.arange(50,110,50)
    order_list = np.arange(2,4)
    epochs_list = [100]

    neurons_list = [5]
    order_list = [2]
    epochs_list = [1]

    lags_list = []
    forecasts = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]

        # Perform grid search
        (_neurons, _order,epochs) = evaluate_multivariate_lstm_models("nested_eval_lstm_multi_oahu", train[neighbor_stations_90], validation[neighbor_stations_90], neurons_list, order_list, epochs_list)

        # Concat train & validation for test
        train = train.append(validation)

        # Perform forecast
        yhat = lstm_multi_forecast(train[neighbor_stations_90], test[neighbor_stations_90], _order, 1, _neurons,epochs)
        
        yhat.append(0) #para manter o formato do vetor de metricas
        
        lags_list.append(_order)
        forecasts.append(yhat)

    return forecasts, lags_list

In [54]:
forecasts, order_list = rolling_cv_lstm_multi(norm_df, 1)

Index:  2010-06-01
LSTM (5, 2, 1)  RMSE=0.424
Best LSTM((5, 2, 1)) RMSE=0.424
Index:  2010-06-08
LSTM (5, 2, 1)  RMSE=0.411
Best LSTM((5, 2, 1)) RMSE=0.411
Index:  2010-06-15
LSTM (5, 2, 1)  RMSE=0.276
Best LSTM((5, 2, 1)) RMSE=0.276
Index:  2010-06-22
LSTM (5, 2, 1)  RMSE=0.311
Best LSTM((5, 2, 1)) RMSE=0.311
Index:  2010-06-01
LSTM (5, 2, 1)  RMSE=0.476
Best LSTM((5, 2, 1)) RMSE=0.476
Index:  2010-06-08
LSTM (5, 2, 1)  RMSE=0.584
Best LSTM((5, 2, 1)) RMSE=0.584
Index:  2010-06-15
LSTM (5, 2, 1)  RMSE=0.464
Best LSTM((5, 2, 1)) RMSE=0.464
Index:  2010-06-22
LSTM (5, 2, 1)  RMSE=0.437
Best LSTM((5, 2, 1)) RMSE=0.437


In [55]:
forecasts_final = get_final_forecast(forecasts)

In [56]:
calculate_rolling_error("rolling_cv_oahu_raw_lstm_multi", df, forecasts_final, order_list)

## LSTM Univariate

In [57]:
def rolling_cv_lstm_uni(df, step):
    
    neurons_list = np.arange(50,110,50)
    order_list = np.arange(2,4)
    epochs_list = [100]

    neurons_list = [5]
    order_list = [2]
    epochs_list = [1]

    lags_list = []
    forecasts = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]

        # Perform grid search
        (_neurons, _order,epochs) = evaluate_multivariate_lstm_models("nested_eval_lstm_multi_oahu", train[[target_station]], validation[[target_station]], neurons_list, order_list, epochs_list)

        # Concat train & validation for test
        train = train.append(validation)

        # Perform forecast
        yhat = lstm_multi_forecast(train[[target_station]], test[[target_station]], _order, 1, _neurons,epochs)
        
        yhat.append(0) #para manter o formato do vetor de metricas
        
        lags_list.append(_order)
        forecasts.append(yhat)

    return forecasts, lags_list

In [58]:
forecasts, order_list = rolling_cv_lstm_uni(norm_df, 1)

Index:  2010-06-01
LSTM (5, 2, 1)  RMSE=0.646
Best LSTM((5, 2, 1)) RMSE=0.646
Index:  2010-06-08
LSTM (5, 2, 1)  RMSE=0.459
Best LSTM((5, 2, 1)) RMSE=0.459
Index:  2010-06-15
LSTM (5, 2, 1)  RMSE=0.412
Best LSTM((5, 2, 1)) RMSE=0.412
Index:  2010-06-22
LSTM (5, 2, 1)  RMSE=0.581
Best LSTM((5, 2, 1)) RMSE=0.581
Index:  2010-06-01
LSTM (5, 2, 1)  RMSE=0.412
Best LSTM((5, 2, 1)) RMSE=0.412
Index:  2010-06-08
LSTM (5, 2, 1)  RMSE=0.568
Best LSTM((5, 2, 1)) RMSE=0.568
Index:  2010-06-15
LSTM (5, 2, 1)  RMSE=0.423
Best LSTM((5, 2, 1)) RMSE=0.423
Index:  2010-06-22
LSTM (5, 2, 1)  RMSE=0.521
Best LSTM((5, 2, 1)) RMSE=0.521


In [59]:
forecasts_final = get_final_forecast(forecasts)

In [60]:
calculate_rolling_error("rolling_cv_oahu_raw_lstm_uni", df, forecasts_final, order_list)

## MLP Multivariate

In [81]:
def mlp_multi_forecast(train_df, test_df, _order, _steps, _neurons, _epochs):

    
    nfeat = len(train_df.columns)
    nlags = _order
    nsteps = _steps
    nobs = nlags * nfeat
    
    train_reshaped_df = series_to_supervised(train_df, n_in=nlags, n_out=nsteps)
    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
    
    test_reshaped_df = series_to_supervised(test_df, n_in=nlags, n_out=nsteps)
    test_X, test_Y = test_reshaped_df.iloc[:,:nobs].values, test_reshaped_df.iloc[:,-nfeat].values
    
    # design network
    model = designMLPNetwork(_neurons,train_X.shape[1])
    
    # fit network
    model.fit(train_X, train_Y, epochs=_epochs, batch_size=1000, verbose=False, shuffle=False)
    
    forecast = model.predict(test_X)
    
    fcst = [f[0] for f in forecast]

    return fcst

In [82]:
def evaluate_multivariate_mlp_models(test_name, train_df, validation_df, neurons_list, order_list, epochs_list):
    
    lstm_results = pd.DataFrame(columns=['Neurons','Order','Epochs','RMSE'])
    best_score, best_cfg = float("inf"), None
    
    nfeat = len(train_df.columns)
    nsteps = 1
    
    for _neurons in neurons_list:
        for _order in order_list:
            for epochs in epochs_list:
                    
                    nobs = nfeat * _order
                    
                    train_reshaped_df = series_to_supervised(train_df, n_in=_order, n_out=nsteps)
                    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
                    
                    val_reshaped_df = series_to_supervised(validation_df, n_in=_order, n_out=nsteps)
                    validation_X, validation_Y = val_reshaped_df.iloc[:,:nobs].values, val_reshaped_df.iloc[:,-nfeat].values
                   
                    model = designMLPNetwork(_neurons,train_X.shape[1])
                                        
                    # fit network
                    history = model.fit(train_X, train_Y, epochs=epochs, batch_size=1000, verbose=False, shuffle=False)
                    forecast = model.predict(validation_X)
                    fcst = [f[0] for f in forecast]
                    
                    
                    rmse = Measures.rmse(validation_Y, fcst)
                    #rmse = math.sqrt(mean_squared_error(validation_Y, forecast))
                    
                    params = (_neurons, _order,epochs)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, params

                    res = {'Neurons':_neurons, 'Order':_order, 'Epochs' : epochs ,'RMSE' : rmse}
                    print('LSTM %s  RMSE=%.3f' % (params,rmse))
                    lstm_results = lstm_results.append(res, ignore_index=True)
                    lstm_results.to_csv(test_name+".csv")

    print('Best MLP(%s) RMSE=%.3f' % (best_cfg, best_score))
    return best_cfg

In [95]:
def designMLPNetwork(neurons, shape):

    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_dim=shape))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

In [97]:
def rolling_cv_mlp_multi(df, step):
    
    neurons_list = np.arange(50,110,50)
    order_list = np.arange(2,4)
    epochs_list = [100]

    neurons_list = [50]
    order_list = [4]
    epochs_list = [500]

    lags_list = []
    forecasts = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]

        # Perform grid search
        (_neurons, _order,epochs) = evaluate_multivariate_mlp_models("nested_eval_mlp_multi_oahu", train[neighbor_stations_90], validation[neighbor_stations_90], neurons_list, order_list, epochs_list)

        # Concat train & validation for test
        train = train.append(validation)

        # Perform forecast
        yhat = mlp_multi_forecast(train[neighbor_stations_90], test[neighbor_stations_90], _order, 1, _neurons,epochs)
        
        yhat.append(0) #para manter o formato do vetor de metricas
        
        lags_list.append(_order)
        forecasts.append(yhat)

    return forecasts, lags_list

In [98]:
forecasts, order_list = rolling_cv_mlp_multi(norm_df, 1)

Index:  2010-06-01
LSTM (50, 4, 500)  RMSE=0.009
Best MLP((50, 4, 500)) RMSE=0.009
Index:  2010-06-08
LSTM (50, 4, 500)  RMSE=0.008
Best MLP((50, 4, 500)) RMSE=0.008
Index:  2010-06-15
LSTM (50, 4, 500)  RMSE=0.011
Best MLP((50, 4, 500)) RMSE=0.011
Index:  2010-06-22
LSTM (50, 4, 500)  RMSE=0.007
Best MLP((50, 4, 500)) RMSE=0.007
Index:  2010-06-01
LSTM (50, 4, 500)  RMSE=0.110
Best MLP((50, 4, 500)) RMSE=0.110
Index:  2010-06-08
LSTM (50, 4, 500)  RMSE=0.137
Best MLP((50, 4, 500)) RMSE=0.137
Index:  2010-06-15
LSTM (50, 4, 500)  RMSE=0.129
Best MLP((50, 4, 500)) RMSE=0.129
Index:  2010-06-22
LSTM (50, 4, 500)  RMSE=0.115
Best MLP((50, 4, 500)) RMSE=0.115


In [99]:
forecasts_final = get_final_forecast(forecasts)
calculate_rolling_error("rolling_cv_oahu_raw_mlp_multi", df, forecasts_final, order_list)

## MLP Univariate

In [100]:
def rolling_cv_mlp_uni(df, step):
    
    neurons_list = [50]
    order_list = [4]
    epochs_list = [500]

    lags_list = []
    forecasts = []

    limit = df.index[-1].strftime('%Y-%m-%d')

    test_end = ""
    index = df.index[0]

    while test_end < limit :
        print("Index: ", index.strftime('%Y-%m-%d'))  

        train_start, train_end, validation_start, validation_end, test_start, test_end = getRollingWindow(index)
        index = index + datetime.timedelta(days=7)
        
        train = df[train_start : train_end]
        validation = df[validation_start : validation_end]
        test = df[test_start : test_end]

        # Perform grid search
        (_neurons, _order,epochs) = evaluate_multivariate_mlp_models("nested_eval_mlp_uni_oahu", train[[target_station]], validation[[target_station]], neurons_list, order_list, epochs_list)

        # Concat train & validation for test
        train = train.append(validation)

        # Perform forecast
        yhat = mlp_multi_forecast(train[[target_station]], test[[target_station]], _order, 1, _neurons,epochs)
        
        yhat.append(0) #para manter o formato do vetor de metricas
        
        lags_list.append(_order)
        forecasts.append(yhat)

    return forecasts, lags_list

In [101]:
forecasts, order_list = rolling_cv_mlp_uni(norm_df, 1)

Index:  2010-06-01
LSTM (50, 4, 500)  RMSE=0.005
Best MLP((50, 4, 500)) RMSE=0.005
Index:  2010-06-08
LSTM (50, 4, 500)  RMSE=0.010
Best MLP((50, 4, 500)) RMSE=0.010
Index:  2010-06-15
LSTM (50, 4, 500)  RMSE=0.006
Best MLP((50, 4, 500)) RMSE=0.006
Index:  2010-06-22
LSTM (50, 4, 500)  RMSE=0.006
Best MLP((50, 4, 500)) RMSE=0.006
Index:  2010-06-01
LSTM (50, 4, 500)  RMSE=0.103
Best MLP((50, 4, 500)) RMSE=0.103
Index:  2010-06-08
LSTM (50, 4, 500)  RMSE=0.128
Best MLP((50, 4, 500)) RMSE=0.128
Index:  2010-06-15
LSTM (50, 4, 500)  RMSE=0.131
Best MLP((50, 4, 500)) RMSE=0.131
Index:  2010-06-22
LSTM (50, 4, 500)  RMSE=0.110
Best MLP((50, 4, 500)) RMSE=0.110


In [102]:
forecasts_final = get_final_forecast(forecasts)
calculate_rolling_error("rolling_cv_oahu_raw_mlp_uni", df, forecasts_final, order_list)